In [1]:
import os 
from datetime import date
from matplotlib import pyplot as plt
import numpy as np

# we need to set the current path to the directory
# containing the suite3d repository, this hack should
# do the trick
os.chdir(os.path.dirname(os.path.abspath("")))

from suite3d.job import Job
from suite3d import io
from suite3d import plot_utils as plot

In [2]:
# update this to point to the demo data! 
tifs = io.get_tif_paths('/mnt/md0/data/demo/lbm')[:10]
for tif in tifs: print(tif)

/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00001.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00002.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00003.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00004.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00005.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00006.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00007.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00008.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00009.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00010.tif


In [3]:
job_params = {    
    # number of channels recorded in the tiff file, typically 30
    'n_ch_tif' : 26,
    
    # number of planes in the deeper cavity, typically 15
    'cavity_size' : 13,

    # convert from the Scanimage channel ordering to deep-to-shallow ordering
    # in our case, ScanImage channel numbers are in temporal order. 
    'planes' : np.array([ 0,  2,  4,  6,  8, 10, 12, 14,
                        16, 18, 20, 22, 24, 1,  3, 5,  7,9,11, 13, 15,17]),

    # voxel size in z,y,x in microns
    'voxel_size_um' : (20, 2.5, 2.5),

    # number of files to use for the initial pass
    # usually, ~500 frames is a good rule of thumb
    # we will just use 200 here for speed
    'n_init_files' :  2,
    
    # number of pixels to fuse between the ROI strips
    'fuse_shift_override' : 7,   
    # will try to automatically estimate crosstalk using 
    # the shallowest crosstalk_n_planes planes. if you want to override,
    # set override_crosstalk = float between 0 and 1
    'subtract_crosstalk' : True,
    
    # volume rate in acquisition
    'fs' : io.get_si_params(tifs[0])['vol_rate'],

    'tau' : 1.3, # approx half-life of GCamP6s
    
    # 3D GPU registration - fast! 
    '3d_reg' : True,
    'gpu_reg' : True,
    # note : 3D CPU is not supported yet
}

In [4]:
# Create the job
job = Job('/mnt/md0/runs','demo', tifs = tifs,
          params=job_params, create=True, overwrite=True, verbosity = 1)

Job directory /mnt/md0/runs/s3d-demo already exists
Loading job directory for demo in /mnt/md0/runs
   Loading dirs 
   Loading default params
   Updated main params file


In [5]:
job.run_init_pass()

   Saved a copy of params at /mnt/md0/runs/s3d-demo/summary
   Updated main params file
Launching initial pass
Saving summary to /mnt/md0/runs/s3d-demo/summary/summary.npy
   Loading init tifs with 26 channels
    Loaded file into shared memory in 3.65 sec
    Workers completed in 0.88 sec
    Total time: 4.52 sec
    Loaded file into shared memory in 3.62 sec
    Workers completed in 0.94 sec
    Total time: 4.56 sec
   Loaded 2 files, total 5.90 GB
   Not enough frames in loaded tifs - using 200 init frames instead
   Loaded movie with 200 frames and shape 22, 1000, 720
Estiamted crosstalk in 7.957156419754028s
   Subtracting with estimated coefficient 0.245
   Using 3d registration
   Computing plane alignment shifts
   Applying plane alignment shifts
   Launching 3D GPU reference image calculation
   Saved init mov to /mnt/md0/runs/s3d-demo/summary/init_mov.npy
   Saving summary to /mnt/md0/runs/s3d-demo/summary/summary.npy
   Initial pass complete. See /mnt/md0/runs/s3d-demo/summa

In [6]:
# OPTIONAL: load and take a look at the reference image
summary = job.load_summary()
ref_img = summary['ref_img_3d']

# view 1 plane at a time
# plot.show_img(ref_img[5], figsize=(3,4))

# interactive 3D viewer
# plot.VolumeViewer(ref_img)


In [7]:
job.register()

   Saved a copy of params at /mnt/md0/runs/s3d-demo/registered_fused_data
   Updated main params file
   Starting registration: 3D: True, GPU: True
   Overriding fuse shift value to 7
   Not clipping frames for registration
Will analyze 10 tifs in 10 batches
   Enforcing positivity
   Launching IO thread
Loading Batch 0 of 9
    Loaded file into shared memory in 3.59 sec
    Workers completed in 1.69 sec
    Total time: 5.28 sec
   Loaded 1 files, total 2.95 GB
   Batch 0 IO thread joined
   Launching IO thread for next batch
    Loaded file into shared memory in 4.74 sec
    Workers completed in 1.74 sec
    Total time: 6.49 sec
   Completed rigid reg on batch in :8.487227201461792s
   Loaded 1 files, total 2.95 GB
   Shifted the mov in: 1.8406500816345215s
Loading Batch 1 of 9
   Batch 1 IO thread joined
   Launching IO thread for next batch
    Loaded file into shared memory in 3.73 sec
    Workers completed in 2.24 sec
    Total time: 5.97 sec
   Loaded 1 files, total 2.95 GB
   Co

In [5]:
job.verbosity = 10

In [6]:
job.params['detection_timebin'] = None

In [7]:
corr_map = job.calculate_corr_map()

      Found dir /mnt/md0/runs/s3d-demo/corrmap
      Updating self.dirs tag corrmap
      Found dir /mnt/md0/runs/s3d-demo/mov_sub
      Updating self.dirs tag mov_sub
   Updated detection_timebin to 6 based on framerate and tau
   Saved a copy of params at /mnt/md0/runs/s3d-demo/corrmap
   Updated main params file
   Computing correlation map of movie with 1000 frames, volume shape: 22, 1041, 737
      Running batch 1 of 2
      Binning with timebin of size 06
         Timer batch_timebin completed in 0.002 sec
         Timer prep completed in 16.483 sec
         Timer batch_setup completed in 2.286 sec
         Timer batch_edgecrop completed in 0.190 sec
         Timer accum_meanmeax completed in 2.291 sec
         Timer batch_rolling_mean_filt completed in 2.056 sec
            THIS IS A BOTTLENECK - parallelize
         Timer batch_accum_sdmov completed in 5.816 sec
         Timer batch_norm_sdmov completed in 0.989 sec
         Loading movie of size (133, 22, 1041, 737) into share

/home/ali/anaconda3/envs/s3d-pip/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 2 leaked shared_memory objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/home/ali/anaconda3/envs/s3d-pip/lib/python3.11/multiprocessing/resource_tracker.py:267: UserWarning: resource_tracker: '/psm_475d0968': [Errno 2] No such file or directory: '/psm_475d0968'
  warnings.warn('resource_tracker: %r: %s' % (name, e))
/home/ali/anaconda3/envs/s3d-pip/lib/python3.11/multiprocessing/resource_tracker.py:267: UserWarning: resource_tracker: '/psm_92476926': [Errno 2] No such file or directory: '/psm_92476926'
  warnings.warn('resource_tracker: %r: %s' % (name, e))
/home/ali/anaconda3/envs/s3d-pip/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 2 leaked shared_memory objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/h

         Timer batch_setup completed in 0.583 sec
         Timer batch_edgecrop completed in 0.045 sec
         Timer accum_meanmeax completed in 0.632 sec
         Timer batch_rolling_mean_filt completed in 0.432 sec
            THIS IS A BOTTLENECK - parallelize
         Timer batch_accum_sdmov completed in 1.390 sec
         Timer batch_norm_sdmov completed in 0.230 sec
         Loading movie of size (33, 22, 1041, 737) into shared memory
         Timer dtu_shmem completed in 1.774 sec
         Subtracting neuropil and applying cell filters
         Timer dtu_npsub_conv3d completed in 10.563 sec
         Reducing filtered movie to compute correlation map
         Timer dtu_vmap completed in 0.615 sec
         Timer dtu_cleanup completed in 0.924 sec
         Timer batch_filt_reduce completed in 13.877 sec
         Timer batch_accum_vmap completed in 0.084 sec
         Timer batch completed in 17.666 sec
         Timer save completed in 1.755 sec


In [8]:
# you can load the vmap and visualize it at this stage, using the same plot.___ functions as above
# res = job.load_corr_map_results()
# vmap = res['vmap']

In [9]:
# since we have a short movie, we can fit a larger patch into memory
# so we increase this from (150,150) to (250,250)
job.params['patch_size_xy'] = (250, 250)
# for speed, only segment a single patch
job.segment_rois(patches_to_segment = (5,))

      Found dir /mnt/md0/runs/s3d-demo/segmentation
      Updating self.dirs tag segmentation
   Saved a copy of params at /mnt/md0/runs/s3d-demo/segmentation
   Updated main params file
      Found dir /mnt/md0/runs/s3d-demo/rois
      Updating self.dirs tag rois
   Saving results to /mnt/md0/runs/s3d-demo/segmentation and /mnt/md0/runs/s3d-demo/rois 
dict_keys(['max_img', 'mean_img', 'vmap', 'vmap_raw', 'all_params'])
   Detecting from patch 1 / 1
      Found dir /mnt/md0/runs/s3d-demo/segmentation/patch-0005
         Loading 0.43 GB movie to memory, shape: (166, 22, 250, 250) 
         Loaded
         Loading movie patch to shared memory
         Loaded
      Starting extraction with peak_thresh: 0.100 and Th2: 5.000
         Iter 0000: running 16 ROIs in parallel
         Added cell 1 at 13, 309, 269, peak: 1.731, thresh: 5.000, 43 frames, 680 pixels
         Added cell 2 at 03, 295, 323, peak: 1.720, thresh: 5.000, 37 frames, 558 pixels
         Added cell 3 at 06, 294, 264, peak:

'/mnt/md0/runs/s3d-demo/rois'

In [11]:
job.compute_npil_masks()
traces = job.extract_and_deconvolve()

   Updated main params file
   Movie shape: (22, 1000, 1041, 737)
790
   Extracting 790 valid cells, and saving cell flags to /mnt/md0/runs/s3d-demo/rois/iscell_extracted.npy
   Extracting activity
   Saving intermediate results to /mnt/md0/runs/s3d-demo/rois
   Deconvolving
   Saving to /mnt/md0/runs/s3d-demo/rois


In [13]:
from suite3d import ui, plot_utils

In [14]:
seg_results = job.load_segmentation_results()

corrmap_vol = seg_results['info']['vmap']
mean_vol = seg_results['info']['mean_img']
cell_vol = ui.make_label_vols(seg_results['stats'],corrmap_vol.shape)

plot_utils.VolumeViewer(corrmap_vol, overlay=cell_vol[1]) 

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [ ]:
job.export_results('path/to/output',result_dir_name='rois')

To take a look at the outputs in napari, navigate to the suite3d directory in a command shell and run the following:
```
python curation.py curation --output_dir /path/to/output/rois
```